In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dbb = pd.read_csv("time_fixed_DB.csv", encoding="UTF-8",sep=";")
db = dbb.loc[dbb["userId"] < 8e3].copy()
dff= pd.read_csv("time_fixed_DF.csv", encoding="UTF-8",sep=";")

In [1]:
matchedUsers = {}  # empty dictionary to be filled by the matched users (key:value) = (movieLensID,netflixID : eccentricity)
db.set_index(["movieId"],inplace=True)  # rename the column
weight = 1/np.log10(dff.loc[:,"movieId"].value_counts())  # apply the weights beforehand to aviod log(1) and computation efficiency

for i in range(1,21): # for loop in order to grab subsets from netflix data
    print(i)
    upperBound = 1000*i # upper bound for netflix subset
    lowerBound = upperBound-1000 # lower bound for netflix subset
    df = dff.loc[(dff["userId"] > lowerBound) & (dff["userId"] < upperBound) ].copy() # grab the subset

    db.rename(columns={"userId":"uM"},inplace=1) # renaming the columns
    df.rename(columns={"userId":"uF"},inplace=1)
    df.set_index(["movieId"],inplace=True)

    com_movies = df.index.intersection(db.index).unique() # ordering and finding the intersections between the two data sets
    db = db.loc[com_movies,:] # corresponding common movies from movielens
    df = df.loc[com_movies,:] # corresponding common movies from netflix
    res = df.subtract(db,fill_value=0).astype(np.int64)
    res = np.abs(res)
    result = weight.loc[com_movies]*(np.exp(-res.loc[:,'rating']/1.5) + np.exp(-res.loc[:,'timestamp']/30)) # rating algorithm

    res.loc[:,"score"] = result # renaming things
    res.drop(["rating", "timestamp"],axis=1,inplace=True)
    res.set_index(['uF','uM'],inplace=True)

    R = res.groupby(level=[0,1]).sum()

    values = R.index.levels[0]

    for elem in values: # calculate the eccentricity between the two highest scores
        tempUser = R.loc[elem]
        tempScore = np.array(tempUser["score"])
        stdev = np.std(tempScore)
        best1 = np.max(tempScore)
        idx1 = np.argmax(tempScore)
        tempScore = np.delete(tempScore, idx1)
        best2 = np.max(tempScore)
        ecc = (best1-best2)/stdev
        if (ecc >= 1.5): # if the eccentricity is higher than the given threshold (1.5 in this case) that is match. Be happy!
            matchedUsers[elem,idx1] = ecc


NameError: name 'db' is not defined

In [20]:
import pickle
with open('usersFound.pkl','wb') as f: # save the data as pickle format
    pickle.dump(matchedUsers,f)